In [1]:
import numpy as np
import pymetis
import pickle

In [2]:
data_dir = '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/'
clean_data_dir = '../../data/chips/clean_data/'
n_variants = 13

for i in range(1, n_variants+1):
    connection_data = np.load(f'{data_dir}{i}/xbar_connectivity.npz')

    # Number of nodes and hyperedges
    num_nodes = max(connection_data['row']) + 1
    num_nets = max(connection_data['col']) + 1
    num_partitions = int(np.sqrt(num_nodes))

    # Step 1: Create an adjacency list for Metis
    # Convert hypergraph to bipartite graph representation
    adj_list = [[] for _ in range(num_nodes + num_nets)]
    for node, net in zip(connection_data['row'], connection_data['col']):
        adj_list[node].append(num_nodes + net)  # Connect node to net
        adj_list[num_nodes + net].append(node)  # Connect net to node

    # Step 2: Run Metis partitioning
    cuts, membership = pymetis.part_graph(num_partitions, adjacency=adj_list)
    # dct = {'partition': np.array(membership[:num_nodes])}
    arr = np.array(membership[:num_nodes])

    # with open(f'{clean_data_dir}{i}.partition.pkl', 'wb') as f:
    #     pickle.dump(dct, f)
    np.save(f'{clean_data_dir}{i}.partition.npy', arr)